In [2]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt

In [11]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

# Load images from dataset
def load_images_from_folder(folder, target_size=(256, 256)):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, target_size)  # Resize images to a uniform size
            images.append(img)
    return images

# Add noise to images
def add_noise(images, noise_factor=0.5):
    noisy_images = []
    for img in images:
        noise = np.random.normal(loc=0.0, scale=noise_factor, size=img.shape)
        noisy_img = np.clip(img + noise, 0., 255.)
        noisy_images.append(noisy_img.astype(np.uint8))
    return noisy_images

# Build a simple Denoising Autoencoder
def build_denoising_autoencoder(input_shape):
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))
    model.add(layers.Conv2D(16, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), padding='same'))
    model.add(layers.Conv2D(8, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), padding='same'))
    model.add(layers.Conv2D(8, (3, 3), activation='relu', padding='same'))
    model.add(layers.UpSampling2D((2, 2)))
    model.add(layers.Conv2D(16, (3, 3), activation='relu', padding='same'))
    model.add(layers.UpSampling2D((2, 2)))
    model.add(layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same'))
    return model

# Denoise images
def denoise_images(model, noisy_images):
    noisy_images = np.array(noisy_images) / 255.0  # Normalize
    denoised_images = model.predict(noisy_images)
    return (denoised_images * 255).astype(np.uint8)

# Save images to folder
def save_images(images, folder):
    if not os.path.exists(folder):
        os.makedirs(folder)
    for i, img in enumerate(images):
        cv2.imwrite(os.path.join(folder, f'denoised_image_{i}.png'), img)

# Main function
def main():
    # Paths
    dataset_path = 'covid'
    save_path = 'denoised_images'

    # Load images
    images = load_images_from_folder(dataset_path)

    # Add noise
    noisy_images = add_noise(images)

    # Build and compile the denoising autoencoder model
    input_shape = (images[0].shape[0], images[0].shape[1], 1)  # Assuming images are grayscale
    autoencoder = build_denoising_autoencoder(input_shape)
    autoencoder.compile(optimizer='adam', loss='mse')

    # Train the autoencoder (for demo purposes; adjust epochs and batch size as necessary)
    # Note: This is a simple example. For best results, you should train on a larger dataset.
    x_train = np.array(noisy_images) / 255.0  # Normalize
    x_train_clean = np.array(images) / 255.0  # Normalize
    autoencoder.fit(x_train, x_train_clean, epochs=10, batch_size=32)

    # Denoise images
    denoised_images = denoise_images(autoencoder, noisy_images)

    # Save denoised images
    save_images(denoised_images, save_path)

    print(f"Denoised images saved to '{save_path}'.")

if __name__ == "__main__":
    main()


Epoch 1/10
5/5 [==============================] - 3s 304ms/step - loss: 0.0637
Epoch 2/10
5/5 [==============================] - 2s 308ms/step - loss: 0.0605
Epoch 3/10
5/5 [==============================] - 1s 279ms/step - loss: 0.0563
Epoch 4/10
5/5 [==============================] - 1s 271ms/step - loss: 0.0501
Epoch 5/10
5/5 [==============================] - 2s 313ms/step - loss: 0.0439
Epoch 6/10
5/5 [==============================] - 2s 398ms/step - loss: 0.0360
Epoch 7/10
5/5 [==============================] - 1s 276ms/step - loss: 0.0247
Epoch 8/10
5/5 [==============================] - 2s 287ms/step - loss: 0.0140
Epoch 9/10
5/5 [==============================] - 2s 292ms/step - loss: 0.0114
Epoch 10/10
5/5 [==============================] - 1s 75ms/step
Denoised images saved to 'denoised_images'.


In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

# Load images from dataset
def load_images_from_folder(folder, target_size=(256, 256)):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, target_size)  # Resize images to a uniform size
            images.append(img)
    return images

# Add noise to images
def add_noise(images, noise_factor=0.5):
    noisy_images = []
    for img in images:
        noise = np.random.normal(loc=0.0, scale=noise_factor, size=img.shape)
        noisy_img = np.clip(img + noise, 0., 255.)
        noisy_images.append(noisy_img.astype(np.uint8))
    return noisy_images

# Build U-Net model
def build_unet(input_shape):
    inputs = layers.Input(shape=input_shape)

    # Encoder
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    # Bottleneck
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)

    # Decoder
    u5 = layers.UpSampling2D((2, 2))(c4)
    u5 = layers.concatenate([u5, c3])
    c5 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u5)
    c5 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c5)

    u6 = layers.UpSampling2D((2, 2))(c5)
    u6 = layers.concatenate([u6, c2])
    c6 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u6)
    c6 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c6)

    u7 = layers.UpSampling2D((2, 2))(c6)
    u7 = layers.concatenate([u7, c1])
    c7 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u7)
    c7 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c7)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c7)

    model = models.Model(inputs=[inputs], outputs=[outputs])
    return model

# Denoise images
def denoise_images(model, noisy_images):
    noisy_images = np.array(noisy_images) / 255.0  # Normalize
    denoised_images = model.predict(noisy_images)
    return (denoised_images * 255).astype(np.uint8)

# Save images to folder
def save_images(images, folder):
    if not os.path.exists(folder):
        os.makedirs(folder)
    for i, img in enumerate(images):
        cv2.imwrite(os.path.join(folder, f'denoised_image_{i}.png'), img)

# Main function
def main():
    # Paths
    dataset_path = 'covid'
    save_path = 'denoised_images_unet'

    # Load images
    images = load_images_from_folder(dataset_path)

    # Add noise
    noisy_images = add_noise(images)

    # Build and compile the U-Net model
    input_shape = (images[0].shape[0], images[0].shape[1], 1)  # Assuming images are grayscale
    unet_model = build_unet(input_shape)
    unet_model.compile(optimizer='adam', loss='mse')

    # Train the U-Net (for demo purposes; adjust epochs and batch size as necessary)
    x_train = np.array(noisy_images) / 255.0  # Normalize
    x_train_clean = np.array(images) / 255.0  # Normalize
    unet_model.fit(x_train, x_train_clean, epochs=50, batch_size=16)

    # Denoise images
    denoised_images = denoise_images(unet_model, noisy_images)

    # Save denoised images
    save_images(denoised_images, save_path)

    print(f"Denoised images saved to '{save_path}'.")

if __name__ == "__main__":
    main()






Epoch 1/50

